In [2]:
# %%capture
# !pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

In [3]:
import os
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date
import datetime as dt
import pandas as pd
import sys
import csv
import numpy as np
import multiprocessing
import re
from collections import namedtuple
# import py4j
import findspark
findspark.init()
import pyspark
os.environ['PYSPARK_SUBMIT_ARGS'] = """--name job_name --master local --conf spark.dynamicAllocation.enabled=true pyspark-shell"""
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_251.jdk/Contents/Home"

In [ ]:
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
from pymetamap import MetaMap

In [5]:
import psycopg2
import pandas.io.sql as sqlio
# from tabulate import tabulate

In [ ]:
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
  
from pymetamap import MetaMap

In [34]:
import json
import argparse
import lxml
import config
import pickle

# from Authentication import *
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [ ]:
url = "https://aact.ctti-clinicaltrials.org/pipe_files"
response = requests.get(url)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text)
# print(soup.prettify())

In [ ]:

# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220301_pipe-delimited-export.zip
# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220801_pipe-delimited-export.zip
upload_dates = []
zip_files = []
links = []
body = soup.find_all('td', attrs={'class': 'file-archive'}) #Find all
for el in body:
    tags = el.find('a')
    try:
        if 'href' in tags.attrs:   # looking for href inside anchor tag    
            link = "https://aact.ctti-clinicaltrials.org" + tags.get('href')
            links.append(link)
            last_upload = link.split("/")[-1]
            zip_files.append(last_upload)
            date_upload = last_upload.split("_")[0]
            upload_dates.append(date_upload)    # appending link to list of links
    except:    # pass if list missing anchor tag or anchor tag does not has a href params 
        pass
    
print(upload_dates)

upload_dates = [dt.datetime.strptime(date, '%Y%m%d').date() for date in upload_dates] # convert all strings in list into datetime objects


In [ ]:
most_recent_date = max(upload_dates)
# print(most_recent_date)

today = dt.date.today()

# UNCOMMENT LINE BELOW WHEN IN PRODUCTION, THIS IS FOR TESTING PURPOSES
most_recent_date = min(upload_dates)

# fix this to run entire script should the current date be 
if most_recent_date < today:
#     date_file_df = pd.DataFrame(list(zip(upload_dates, zip_files, links)))
#     print(links[0])
    resp = urlopen(links[0])
else:
    print("Local instance of clinical trials data is already up to date.")



In [ ]:
pd.set_option('display.max_colwidth', 3000)
# date_file_df[2]

In [ ]:
os.getcwd()


In [6]:
# connect to DB and get the column names of the table
con = None
con = psycopg2.connect(database="aact")
con.rollback()
cursor = con.cursor()

con.autocommit = True # SQL statement is treated as a transaction and is automatically committed right after it is executed
# grab the conditions
sql = '''SELECT * FROM ctgov.conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_conditions
sql = '''SELECT * FROM ctgov.browse_conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the interventions
sql = '''SELECT * FROM ctgov.interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
interventions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_interventions
sql = '''SELECT * FROM ctgov.browse_interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_interventions_df = pd.DataFrame(tuples, columns=column_names)

con.close()


In [7]:
interventions_df = interventions_df.rename(columns={'id': 'int_id',
                                                    'nct_id': 'int_nctid',
                                                    'intervention_type': 'int_type',
                                                    'name': 'int_name',
                                                    'description': 'int_description'})
interventions_df

,int_id,int_nctid,int_type,int_name,int_description
0,4547568,NCT01340339,Device,Phototherapy,Fluorescent reverse phototherapy (7 white ligh...
1,4547569,NCT01340339,Device,Phototherapy,super LED reverse phototherapy (17 bulbs arran...
2,4547570,NCT01340365,Behavioral,Tai Chi Exercise,Practicing Tai Chi exercise 4 times a week for...
3,4547571,NCT01340365,Behavioral,Tai Chi,"Usual care, individuals attend testing session..."
4,4547572,NCT01340391,Device,Omnicast,dorsal splint 2-5 weeks
...,...,...,...,...,...
685271,4139681,NCT03104712,Other,Rice + Pesto,50g available carbohydrate
685272,4139682,NCT03104465,Behavioral,Mindfulness Training,interactive. web-based mindfulness training co...
685273,4139683,NCT03104361,Biological,Platelet-Rich Plasma,Patients who meet all eligible requirements fo...
685274,4139684,NCT03104283,Drug,Apatinib,take apatinib orally until disease progression...


In [8]:
interventions_df = interventions_df.drop(columns=['int_id', 'int_description'])

In [9]:
conditions_df = conditions_df.rename(columns={'id': 'con_id',
                                              'nct_id': 'con_nctid',
                                              'name': 'con_name',
                                              'downcase_name': 'con_downcase_name'})
conditions_df

,con_id,con_nctid,con_name,con_downcase_name
0,3072834,NCT03254264,Autism Spectrum Disorder,autism spectrum disorder
1,3072835,NCT03254329,Human Milk Nutrient Reference Values,human milk nutrient reference values
2,3349485,NCT03460652,ADHD,adhd
3,3349486,NCT03460899,Diabetes Mellitus With Hypoglycemia,diabetes mellitus with hypoglycemia
4,3072836,NCT03254342,Major Depressive Disorder,major depressive disorder
...,...,...,...,...
688783,3072052,NCT03261687,Pregnancy Related,pregnancy related
688784,3072053,NCT03261687,Pelvic Girdle Pain,pelvic girdle pain
688785,3072054,NCT03261622,Faecal Incontinence,faecal incontinence
688786,3072055,NCT03261622,Fecal Incontinence,fecal incontinence


In [10]:
conditions_df = conditions_df.drop(columns=['con_id', 'con_name'])

In [11]:
browse_interventions_df = browse_interventions_df.rename(columns={'id': 'browseint_id',
                                                                  'nct_id': 'browseint_nctid',
                                                                  'mesh_term': 'browseint_meshterm',
                                                                  'downcase_mesh_term': 'browseint_meshterm_downcase',
                                                                  'mesh_type': 'browseint_meshtype'})
browse_interventions_df

,browseint_id,browseint_nctid,browseint_meshterm,browseint_meshterm_downcase,browseint_meshtype
0,4935968,NCT04327843,Haloperidol,haloperidol,mesh-list
1,4935969,NCT04327843,Haloperidol decanoate,haloperidol decanoate,mesh-list
2,4935970,NCT04327843,Antiemetics,antiemetics,mesh-ancestor
3,5198960,NCT00808223,Alefacept,alefacept,mesh-list
4,5198961,NCT00808223,Dermatologic Agents,dermatologic agents,mesh-ancestor
...,...,...,...,...,...
1335262,4400668,NCT03100786,"Antineoplastic Agents, Hormonal","antineoplastic agents, hormonal",mesh-ancestor
1335263,4400669,NCT03100786,Antineoplastic Agents,antineoplastic agents,mesh-ancestor
1335264,4400670,NCT03100565,Lidocaine,lidocaine,mesh-list
1335265,4400671,NCT03100565,"Anesthetics, Local","anesthetics, local",mesh-ancestor


In [12]:
browse_interventions_df = browse_interventions_df.drop(columns=['browseint_id', 'browseint_meshterm'])

In [13]:
browse_conditions_df = browse_conditions_df.rename(columns={'id': 'browsecon_id',
                                                            'nct_id': 'browsecon_nctid',
                                                            'mesh_term': 'browsecon_meshterm',
                                                            'downcase_mesh_term': 'browsecon_meshterm_downcase',
                                                            'mesh_type': 'browsecon_meshtype'})
browse_conditions_df

,browsecon_id,browsecon_nctid,browsecon_meshterm,browsecon_meshterm_downcase,browsecon_meshtype
0,9631397,NCT04035564,Hyponatremia,hyponatremia,mesh-list
1,9631398,NCT04035564,Water-Electrolyte Imbalance,water-electrolyte imbalance,mesh-ancestor
2,9631399,NCT04035564,Metabolic Diseases,metabolic diseases,mesh-ancestor
3,9631400,NCT04032652,Colitis,colitis,mesh-list
4,9631401,NCT04032652,"Colitis, Ulcerative","colitis, ulcerative",mesh-list
...,...,...,...,...,...
2573556,8601072,NCT03213899,Disease Attributes,disease attributes,mesh-ancestor
2573557,8601073,NCT03213899,Pathologic Processes,pathologic processes,mesh-ancestor
2573558,8601074,NCT03214016,Hypertension,hypertension,mesh-list
2573559,8601075,NCT03214016,Vascular Diseases,vascular diseases,mesh-ancestor


In [14]:
browse_conditions_df = browse_conditions_df.drop(columns=['browsecon_id', 'browsecon_meshterm'])

In [15]:
print(len(interventions_df['int_name'].unique()))
print(len(conditions_df['con_downcase_name'].unique()))
print(len(browse_conditions_df['browsecon_meshterm_downcase'].unique()))
print(len(browse_interventions_df['browseint_meshterm_downcase'].unique()))

350361
95250
4280
3744


In [69]:
conditions_df


,con_nctid,con_downcase_name
0,NCT03254264,autism spectrum disorder
1,NCT03254329,human milk nutrient reference values
2,NCT03460652,adhd
3,NCT03460899,diabetes mellitus with hypoglycemia
4,NCT03254342,major depressive disorder
...,...,...
688783,NCT03261687,pregnancy related
688784,NCT03261687,pelvic girdle pain
688785,NCT03261622,faecal incontinence
688786,NCT03261622,fecal incontinence


In [16]:
df = pd.merge(conditions_df, interventions_df, left_on='con_nctid', right_on = 'int_nctid')

In [71]:
df.shape

(1256248, 5)

In [17]:
df_dedup = df.drop_duplicates(subset = ['con_downcase_name', 'int_name'],
                                      keep = 'first').reset_index(drop = True)

In [ ]:
df_dedup.shape

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_dedup.head(1000))

In [ ]:
df_dedup[['con_downcase_name', 'int_name']].to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


# notes on creating edge file

go to /proj/arivale/gglusman/KGs

once there look into wellness/source/wellness_kg_edges_v1.6.tsv

notice the first line: subject	predicate	object	relation	subject_name	object_name	category	N	Type_of_relationship	Strength_of_relationship	qualifiers	qualifier_value	Bonferroni_pval

subject, predicate, object are required (maybe some others, don’t remember)

…and they should be CURIEs

once we map concepts to biolink, mesh etc we’ll have curies

meanwhile, the content you have fits subject_name, object_name

the predicate is what we made up… let’s say ‘evaluated_for’

for short

we’ll have to find a suitable biolink predicate, or suggest a new one

so interventions would be subjects, and conditions would be objects

looking at the several first lines in your file, I see some that make sense, others that seem to suggest there are data problems

for example, “metastatic lung cancer	Pathways” -> Pathways doesn’t seem to be an intervention

‘placebo’ we’ll have to suppress

make sure to include the NCT id in a (non-required) column

it’s essentially part of the provenance

we’ll want to add multiple additional columns with things like FDA approval status, sample size, etc etc etc

look also into EHR/source/ehr_risk_kg_edges_2021_05_07.tsv

…and of course replace ‘edges’ -> ‘nodes’ for their cognate nodes files

wellness/source/wellness_kg_edges_v1.6.tsv -> wellness/source/wellness_kg_nodes_v1.6.tsv

drop "control"

‘control’ is clear, ‘control group’ too… but ‘Control Ostomy Barrier’ is likely not a ‘control’


# My own notes

Use pool/multiprocess to make faster
resolve multiple possible CURIEs per concept

In [18]:
# first get only relevant columns from DB
ct_extract = pd.DataFrame(df_dedup[['con_nctid', 'con_downcase_name', 'int_type', 'int_name']])
ct_extract = ct_extract.rename(columns={'con_nctid': 'nctid'})
# get CURIE column for nct_id column (https://bioregistry.io/registry/clinicaltrials)

ct_extract['nctid_curie'] = ct_extract['nctid']
ct_extract['nctid_curie'] = 'clinicaltrials:' + ct_extract['nctid'].astype(str)

In [23]:
ct_extract

,nctid,con_downcase_name,int_type,int_name,nctid_curie
0,NCT03254264,autism spectrum disorder,Behavioral,Early Start Denver Model,clinicaltrials:NCT03254264
1,NCT03254264,autism spectrum disorder,Behavioral,'as-usual' intervention,clinicaltrials:NCT03254264
2,NCT03254329,human milk nutrient reference values,Other,Assessment of human milk nutrient composition,clinicaltrials:NCT03254329
3,NCT03460652,adhd,Drug,KP415 oral capsule,clinicaltrials:NCT03460652
4,NCT03460899,diabetes mellitus with hypoglycemia,Other,Euglycaemic Clamp,clinicaltrials:NCT03460899
...,...,...,...,...,...
972680,NCT03261687,pelvic girdle pain,Other,water based exercise,clinicaltrials:NCT03261687
972681,NCT03261687,pelvic girdle pain,Other,land based exercise,clinicaltrials:NCT03261687
972682,NCT03261622,faecal incontinence,Device,Sacral nerve stimulation at different stimulat...,clinicaltrials:NCT03261622
972683,NCT03261622,fecal incontinence,Device,Sacral nerve stimulation at different stimulat...,clinicaltrials:NCT03261622


In [ ]:
# let's map all the conditions to BTE
# NOTE: we are attempting to match to BTE terms, but many of the conditions terms are not standardized and are misspelled
# First map as many terms as possible to BTE. Whatever is left over, will use other ontologies to create CURIEs


In [ ]:
# recommedation to first format the output as close to sample as possible

In [20]:
ct_final = pd.DataFrame(columns=['subject','predicate','object','relation','subject_name','object_name','category'])
# no_col_names_df.columns = col_names_df.columns
# chosen predicate = "related_to", bc "associated_with" implies a statistical relationship 
ct_final['subject'] = 'condition:' + ct_extract['con_downcase_name'].astype(str)
ct_final['predicate'] = 'biolink:related_to biolink:clinically_demonstrated_to biolink: hypothesized_to'
ct_final['object'] = 'intervention:' + ct_extract['int_name'].astype(str)   # this will not all be RxNorm CURIEs since some interventions are not drugs
ct_final['relation'] = 'RO:????'
ct_final.subject_name = ct_extract.con_downcase_name
ct_final.object_name = ct_extract.int_name
ct_final.category = 'biolink:Association'
ct_final['nctid'] = ct_extract['nctid']
ct_final['nctid_curie'] = ct_extract['nctid_curie']


In [24]:
ct_final[:1]

,subject,predicate,object,relation,subject_name,object_name,category,nctid,nctid_curie
0,condition:autism spectrum disorder,biolink:related_to biolink:clinically_demonstr...,intervention:Early Start Denver Model,RO:????,autism spectrum disorder,Early Start Denver Model,biolink:Association,NCT03254264,clinicaltrials:NCT03254264


In [ ]:
# interventions will have to be mapped to more than 1 vocabulary unlike conditions, which seems more easily mapped to SNOMED
# first look at the types of interventions (unique on int_type in ct_extract dataframe)

In [25]:
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
  
from pymetamap import MetaMap

# NOTE: YOU MUST START the SKR/Medpost Part-of-Speech Tagger Server and the Word Sense Disambiguation (WSD) Server IN TERMINAL

# % ./bin/skrmedpostctl start
# % ./bin/wsdserverctl start


# stop these servers afterwards



In [22]:
# For pausing
from time import sleep

# Setup UMLS Server
metamap_base_dir = '/gwshare/umls_2021/metamap/public_mm/'
metamap_bin_dir = 'bin/metamap20'
metamap_pos_server_dir = 'bin/skrmedpostctl'
metamap_wsd_server_dir = 'bin/wsdserverctl'

# Start servers
os.system(metamap_base_dir + metamap_pos_server_dir + ' start') # Part of speech tagger
os.system(metamap_base_dir + metamap_wsd_server_dir + ' start') # Word sense disambiguation 

# Sleep a bit to give time for these servers to start up
sleep(60)


In [52]:
# get the directory of the the module
print(os.getcwd())
# Path
path = "/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin"  
# Path of Start directory
start = os.getcwd()
print(os.path.relpath("/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin", os.getcwd()))

/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python
../../../../../../../../Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin


In [ ]:
# For pausing
from time import sleep

# Setup UMLS Server
metamap_base_dir = '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm'
# metamap_bin_dir = 'bin/metamap18'
metamap_pos_server_dir = 'bin/skrmedpostctl'
metamap_wsd_server_dir = 'bin/wsdserverctl'


# Start servers
os.system(metamap_base_dir + metamap_pos_server_dir + ' start') # Part of speech tagger
os.system(metamap_base_dir + metamap_wsd_server_dir + ' start') # Word sense disambiguation 

# Sleep a bit to give time for these servers to start up
sleep(60)


In [27]:
mm = MetaMap.get_instance('/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin/metamap18')

In [29]:
sents = ['myocardial infarction']  # this has to be 1 sentence
concepts,error = mm.extract_concepts(sents)
for concept in concepts:
    print(concept)
    print("\n")

In [82]:
error

In [90]:
# check char encoding, is ascii??
    
for ct_string in ct_final['subject_name'][:5]:
#     print(type(ct_string))
    print(ct_string.isascii())
    

True
True
True
True
True


In [91]:
diseases = list(set(list(ct_final['subject_name'])))
interventions = list(filter(None, ct_final['object_name'].unique()))

In [92]:
diseases[:2]

['osteopetrosis', 'urinary incontinence']

In [55]:
# remove non-ASCII char! MetaMap does not like non-ASCII...

import re
non_ascii = "[^\x00-\x7F]"
pattern = re.compile(r"[^\x00-\x7F]")


def custom_make_translation(text, translation):
    non_ascii_text = re.sub(pattern, ' ', text)
    return non_ascii_text


text = "split-virion, non-adjuvanted H1N1 vaccine of 15 μg"
translated = custom_make_translation(text, ascii_char_dict)

NameError: name 'ascii_char_dict' is not defined

In [93]:
import re

non_ascii = "[^\x00-\x7F]"
pattern = re.compile(r"[^\x00-\x7F]")

def custom_make_translation(text):
    non_ascii_text = re.sub(pattern, ' ', text)
    return non_ascii_text


text = "split-virion, non-adjuvanted H1N1 vaccine of 15 μg"
translated = custom_make_translation(text)

In [94]:
translated

'split-virion, non-adjuvanted H1N1 vaccine of 15  g'

In [95]:
diseases_translated = dict((i, custom_make_translation(i)) for i in diseases)
interventions_translated = dict((i, custom_make_translation(i)) for i in interventions)

In [33]:
# diseases_translated = dict((i, custom_make_translation(i)) for i in diseases[:20])
# interventions_translated = dict((i, custom_make_translation(i)) for i in interventions[:20])

In [173]:
# def run_metamap(input_term, source_restriction):
#     concepts_dict = dict()
#     try:
#         concepts,error = mm.extract_concepts(sentences = [input_term], restrict_to_sources = source_restriction)
#         concepts_dict[input_term] = concepts
#     except:
#         concepts_dict[input_term] = None 
#     return(concepts_dict)

In [46]:
def run_metamap_orig(input_term):
    concepts_dict = dict()
    try:
        concepts,error = mm.extract_concepts(sentences = [input_term])
        concepts_dict[input_term] = concepts
    except:
        concepts_dict[input_term] = None 
    return(concepts_dict)

In [87]:
def run_metamap(input_term, source_restriction):
    concepts_dict = dict()
    if all(x is None for x in source_restriction):
        try:
            concepts,error = mm.extract_concepts([input_term],
                                                 word_sense_disambiguation = True,
                                                 prune = 10,
                                                 composite_phrase = 1)
            concepts_dict[input_term] = concepts
        except:
            concepts_dict[input_term] = None 
    else:
        try:
            concepts,error = mm.extract_concepts([input_term],
                                                 word_sense_disambiguation = True,
                                                 restrict_to_sources=source_restriction,
                                                 prune = 10,
                                                 composite_phrase = 1)
            concepts_dict[input_term] = concepts
        except:
            concepts_dict[input_term] = None 
    return(concepts_dict)

In [ ]:
import multiprocessing

num_workers = multiprocessing.cpu_count()  

# pool = multiprocessing.Pool(num_workers)
# for task in tasks:
#     pool.apply_async(func, args = (task,))

# pool.close()
# pool.join()

In [88]:
# number_of_processes = 4
# metamapped_diseases = multiprocessing.Pool(number_of_processes).starmap(run_metamap,
#                                                                         zip(list(diseases_translated.values()), [['SNOMEDCT_US', 'SNOMEDCT_VET', 'ICD10CM', 'ICD10CM', 'ICD10PCS', 'ICD9CM', 'ICD9CM']]*len(list(diseases_translated.values()))))
# metamapped_interventions = multiprocessing.Pool(number_of_processes).starmap(run_metamap, 
#                                                                              zip(list(interventions_translated.values()),
#                                                                                  [[None]]*len(list(interventions_translated.values()))))
                                                                             
# Stop MetaMap servers
os.system(metamap_base_dir + metamap_pos_server_dir + ' stop') # Part of speech tagger
os.system(metamap_base_dir + metamap_wsd_server_dir + ' stop') # Word sense disambiguation 

In [69]:
metamapped_diseases[:2]

[{'infertile high-risk patients with polycystic ovaries': [ConceptMMI(index='USER', mm='MMI', score='22.36', preferred_name='Polycystic Ovary Syndrome', cui='C0032460', semtypes='[dsyn]', trigger='["Polycystic ovary syndrome"-tx-1-"polycystic ovaries"-noun-0]', location='TX', pos_info='38/18', tree_codes='C04.182.612.765;C13.351.500.056.630.580.765;C19.391.630.580.765'),
   ConceptMMI(index='USER', mm='MMI', score='12.88', preferred_name='Infertility', cui='C0021359', semtypes='[patf]', trigger='["Infertility"-tx-1-"infertile"-adj-0]', location='TX', pos_info='4/9', tree_codes='C12.294.365;C13.351.500.365'),
   ConceptMMI(index='USER', mm='MMI', score='6.70', preferred_name='Patients', cui='C0030705', semtypes='[podg]', trigger='["Patient"-tx-1-"patients"-noun-0]', location='TX', pos_info='24/8', tree_codes='M01.643'),
   ConceptMMI(index='USER', mm='MMI', score='3.57', preferred_name='High risk', cui='C4319571', semtypes='[fndg]', trigger='["High risk"-tx-1-"high-risk"-noun-0]', locat

In [30]:
with open('metamapped_diseases.txt', 'rb') as f:
    metamapped_diseases = pickle.load(f)
    
with open('metamapped_interventions.txt', 'rb') as f:
    metamapped_interventions = pickle.load(f)

In [128]:
# print(metamapped_diseases[:2])
# print("\n")
# print({kv[0]:kv[1] for i, kv in enumerate(diseases_translated.items()) if i <= 4})

# import org.apache.spark.sql.types.{StringType, StructField, StructType}

schema = StructType(
    Array(
        StructField("original_ct_value", StringType,true),
        StructField("metamap_input", StringType,true),
        StructField("metamap_output_preferred_name", StringType,true),
        StructField("cui", StringType,true),
        StructField("score", StringType,true),
        StructField("semtypes", StringType,true),
        
))

col = ['original_ct_value', 'metamap_input', 'metamap_output_preferred_name', 'cui', 'score', 'semtypes' ]

# dataframe = spark.createDataFrame(data, col)

diseases_df_mapped = spark.createDataFrame(col)


for disease in metamapped_diseases[:10]:
        for key in disease:
            for concept in disease[key]:
                concept = concept._asdict()
    #             concept.get('preferred_name')
                row = [key]
                original_disease = (list(diseases_translated.keys())[list(diseases_translated.values()).index(key)])
                row.append(original_disease)
                row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
                print(row)
                diseases_df_mapped = diseases_df_mapped.union(row)



TypeError: Can not infer schema for type: <class 'str'>

In [ ]:
diseases_translated

In [129]:
# write the mapping results to text files for evaluation 
from collections import namedtuple

with open("metamapped_diseases_qual_control.csv", "w") as file:
    csv.excel.delimiter=','

    writer = csv.writer(file, dialect=csv.excel)
    writer.writerow(['original_ct_value', 'metamap_input', 'metamap_input', 'metamap_cui', 'metamap_score', 'metamap_semtypes'])
    for disease in metamapped_diseases[:100]:
        for key in disease:
            for concept in disease[key]:
                concept = concept._asdict()
                row = [key]
                original_disease = (list(diseases_translated.keys())[list(diseases_translated.values()).index(key)])
                row.append(original_disease)
                row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
                writer.writerow(row)

# with open("metamapped_interventions_qual_control.csv", "w") as file:
#     csv.excel.delimiter=','
#     writer = csv.writer(file, dialect=csv.excel)
#     writer.writerow(['input_intervention', 'metamapped_intervention_name', 'metamap_cui', 'metamap_score', 'metamap_semtypes'])
#     for intervention in metamapped_interventions:
#         for key in intervention:
#             for concept in intervention[key]:
#                 concept = concept._asdict()
#     #             concept.get('preferred_name')
#                 row = [key]
#                 row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
#                 writer.writerow(row)
               


In [139]:
from pyspark.sql import Row
# spark - sparkSession
# sc = spark.sparkContext

# Load a text file and convert each line to a Row.
ct_data = sc.textFile("metamapped_diseases_qual_control.csv")
#Split on delimiters
ct_items = ct_data.map(lambda l: l.split(","))
#Convert to Row
ct_data_struct = ct_items.map(lambda p: Row(original_ct_value=int(p[0]),
                                        metamap_input=p[1],
                                        metamap_cui=p[2],
                                        metamap_score=p[3],
                                        metamap_semtypes=[4]))
for i in ct_data_struct.take(5): print(i)
#convert the RDD to DataFrame

ct_df = spark.createDataFrame(ct_data_struct)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (192.168.0.119 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/rdd.py", line 1562, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/py

In [143]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, ArrayType

schema = StructType(
    Array(
        StructField("original_ct_value", StringType,true),
        StructField("metamap_input", StringType,true),
        StructField("metamap_output_preferred_name", StringType,true),
        StructField("cui", StringType,true),
        StructField("score", StringType,true),
        StructField("semtypes", StringType,true),
        
))

test = spark.read.csv(
    "metamapped_diseases_qual_control.csv", 
    header=True, 
    mode="DROPMALFORMED", 
    schema=schema
)

NameError: name 'array' is not defined

In [136]:
from pyspark.sql import Row
# spark - sparkSession
sc = spark.sparkContext

# Load a text file and convert each line to a Row.
orders = sc.textFile("/practicedata/orders")
#Split on delimiters
parts = orders.map(lambda l: l.split(","))
#Convert to Row
orders_struct = parts.map(lambda p: Row(order_id=int(p[0]), order_date=p[1], customer_id=p[2], order_status=p[3]))
for i in orders_struct.take(5): print(i)
#convert the RDD to DataFrame

orders_df = spark.createDataFrame(orders_struct)

AttributeError: 'NoneType' object has no attribute 'sc'

In [137]:
# import pyspark
from pyspark.sql import Row

spark = SparkSession.builder.master("local[2]") \
                    .appName('clinical_trials_etl') \
                    .getOrCreate()

# # test = spark.createDataFrame(Row(**x) for x in metamapped_diseases[:20]).show(truncate=False)
# for disease in metamapped_diseases:
#         for key in disease:
#             for concept in disease[key]:
#                 concept = concept._asdict()
#                 row = [key]
#                 row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
#                 writer.writerow(row)


In [29]:
metamapped_diseases[:2]

[{'infertile high-risk patients with polycystic ovaries': [ConceptMMI(index='USER', mm='MMI', score='22.36', preferred_name='Polycystic Ovary Syndrome', cui='C0032460', semtypes='[dsyn]', trigger='["Polycystic ovary syndrome"-tx-1-"polycystic ovaries"-noun-0]', location='TX', pos_info='38/18', tree_codes='C04.182.612.765;C13.351.500.056.630.580.765;C19.391.630.580.765'),
   ConceptMMI(index='USER', mm='MMI', score='12.88', preferred_name='Infertility', cui='C0021359', semtypes='[patf]', trigger='["Infertility"-tx-1-"infertile"-adj-0]', location='TX', pos_info='4/9', tree_codes='C12.294.365;C13.351.500.365'),
   ConceptMMI(index='USER', mm='MMI', score='6.70', preferred_name='Patients', cui='C0030705', semtypes='[podg]', trigger='["Patient"-tx-1-"patients"-noun-0]', location='TX', pos_info='24/8', tree_codes='M01.643'),
   ConceptMMI(index='USER', mm='MMI', score='3.57', preferred_name='High risk', cui='C4319571', semtypes='[fndg]', trigger='["High risk"-tx-1-"high-risk"-noun-0]', locat

In [25]:
from pyspark.sql import Row
spark.createDataFrame(Row(**x) for x in mylist).show(truncate=False)

[{'infertile high-risk patients with polycystic ovaries': [ConceptMMI(index='USER', mm='MMI', score='22.36', preferred_name='Polycystic Ovary Syndrome', cui='C0032460', semtypes='[dsyn]', trigger='["Polycystic ovary syndrome"-tx-1-"polycystic ovaries"-noun-0]', location='TX', pos_info='38/18', tree_codes='C04.182.612.765;C13.351.500.056.630.580.765;C19.391.630.580.765'),
   ConceptMMI(index='USER', mm='MMI', score='12.88', preferred_name='Infertility', cui='C0021359', semtypes='[patf]', trigger='["Infertility"-tx-1-"infertile"-adj-0]', location='TX', pos_info='4/9', tree_codes='C12.294.365;C13.351.500.365'),
   ConceptMMI(index='USER', mm='MMI', score='6.70', preferred_name='Patients', cui='C0030705', semtypes='[podg]', trigger='["Patient"-tx-1-"patients"-noun-0]', location='TX', pos_info='24/8', tree_codes='M01.643'),
   ConceptMMI(index='USER', mm='MMI', score='3.57', preferred_name='High risk', cui='C4319571', semtypes='[fndg]', trigger='["High risk"-tx-1-"high-risk"-noun-0]', locat

In [11]:


# pip show pyspark


# import pyspark
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()


In [135]:
spark.stop()

In [21]:


# spark = SparkSession.builder.appName("test").config(
#     "spark.driver.extraJavaOptions",
#     "--add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED",
# ).getOrCreate()

In [ ]:
# link nct ids of studies that have the same disease and intervention pair---multiple studies may have studies the same intervention
# restrict returned CUIs of metamapped concepts???

In [13]:
user = os.environ['USER']

In [14]:
user

'Kamileh'

In [23]:
import subprocess
version = subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)
version

b'java version "1.8.0_251"\nJava(TM) SE Runtime Environment (build 1.8.0_251-b08)\nJava HotSpot(TM) 64-Bit Server VM (build 25.251-b08, mixed mode)\n'

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
mapped